In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Predicting Residential EV Charging Loads using Neural Networks\n",
    "\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Setup - import basic data libraries\n",
    "import numpy as np\n",
    "import pandas as pd"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Task Group 1 - Load, Inspect, and Merge Datasets"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Task 1\n",
    "\n",
    "The file `'datasets/EV charging reports.csv'` contains electric vehicle (EV) charging data. These come from various residential apartment buildings in Norway. The data includes specific user and garage information, plug-in and plug-out times, charging loads, and the dates of the charging sessions.\n",
    "\n",
    "The file is imported to a pandas DataFrame named `ev_charging_reports`.\n",
    "\n",
    "Use the `.head()` method to preview the first five rows."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "ev_charging_reports = pd.read_csv('datasets/EV charging reports.csv')\n",
    "ev_charging_reports.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "<details><summary style=\"display:list-item; font-size:16px; color:blue;\">What is the structure of the dataset?</summary>\n",
    "\n",
    "- **session_ID** - the unique id for each EV charging session\n",
    "- **Garage_ID** - the unique id for the garage of the apartment\n",
    "- **User_ID** - the unique id for each user\n",
    "- **User_private** - 1.0 indicates private charge point spaces and 0.0 indicates shared charge point spaces\n",
    "- **Shared_ID** - the unique id if shared charge point spaces are used\n",
    "- **Start_plugin** - the plug-in date and time in the format (day.month.year hour:minute)\n",
    "- **Start_plugin_hour** - the plug-in date and time rounded to the start of the hour\n",
    "- **End_plugout** - the plug-out date and time in the format (day.month.year hour:minute)\n",
    "- **End_plugout_hour** - the start of the hour of the `End_plugout` hour\n",
    "- **El_kWh** - the charged energy in kWh (charging loads)\n",
    "- **Duration_hours** - the duration of the EV connection time per session\n",
    "- **Plugin_category** - the plug-in time categorized by early/late night, morning, afternoon, and evening\n",
    "- **Duration_category** - the plug-in duration categorized by 3 hour groups\n",
    "- **month_plugin_{month}** - the month of the plug-in session\n",
    "- **weekdays_plugin_{day}** - the day of the week of the plug-in session"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Task 2\n",
    "\n",
    "The file `'datasets/Local traffic distribution.csv'` is imported to a pandas DataFrame named `traffic_reports`. This dataset contains the hourly local traffic density counts at 5 nearby traffic locations. \n",
    "\n",
    "Preview the first five rows."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "traffic_reports = pd.read_csv('datasets/Local traffic distribution.csv')\n",
    "traffic_reports.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "<details><summary style=\"display:list-item; font-size:16px; color:blue;\">What is the structure of the dataset?</summary>\n",
    "\n",
    "- **Date_from** - the starting time in the format (day.month.year hour:minute)\n",
    "- **Date_to** - the ending time in the format (day.month.year hour:minute)\n",
    "- **Location 1 to 5** - contains the number of vehicles each hour at a specified traffic location.\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Task 3\n",
    "\n",
    "We'd like to use the traffic data to help our model. The same charging location may charge at different rates depending on the number of cars being charged, so this traffic data might help the model out.\n",
    "\n",
    "Merge the `ev_charging_reports` and `traffic_reports` datasets together into a Dataframe named `ev_charging_traffic` using the columns:\n",
    "\n",
    "- `Start_plugin_hour` in `ev_charging_reports`\n",
    "- `Date_from` in `traffic_reports`"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "ev_charging_reports = ev_charging_traffic = ev_charging_reports.merge(traffic_reports, \n",
    "                                left_on='Start_plugin_hour', \n",
    "                                right_on='Date_from')\n",
    "\n",
    "ev_charging_traffic.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Task 4\n",
    "\n",
    "Use `.info()` to inspect the merged dataset. "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "ev_charging_traffic.info()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "<details><summary style=\"display:list-item; font-size:16px; color:blue;\">What do we notice about merged dataset under inspection?</summary>\n",
    "\n",
    "We see that there are 39 columns and 6,833 rows in our merged dataset.\n",
    "\n",
    "Some notable things we might have to address:\n",
    "\n",
    "- We expected columns like `El_kWh` and `Duration_hours` to be floats but they are actually object data types.\n",
    "\n",
    "- There are many identifying columns like `session_ID` and `User_ID` that might not be useful for training."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Task Group 2 - Data Cleaning and Preparation"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Task 5\n",
    "\n",
    "We will drop the values that are not needed by our model\n",
    "\n",
    "```py\n",
    "['session_ID', 'Garage_ID', 'User_ID', \n",
    "                'Shared_ID',\n",
    "                'Plugin_category','Duration_category', \n",
    "                'Start_plugin', 'Start_plugin_hour', 'End_plugout', 'End_plugout_hour', \n",
    "                'Date_from', 'Date_to']\n",
    "```"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "drop_columns = ['session_ID', 'Garage_ID', 'User_ID', \n",
    "                'Shared_ID',\n",
    "                'Plugin_category','Duration_category', \n",
    "                'Start_plugin', 'Start_plugin_hour', 'End_plugout', 'End_plugout_hour', \n",
    "                'Date_from', 'Date_to']\n",
    "\n",
    "ev_charging_traffic = ev_charging_traffic.drop(columns=drop_columns, axis=1)\n",
    "ev_charging_traffic.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Task 6\n",
    "\n",
    "Remove the \",\" in both `El_kWh` and `Duration_hours` and replace it by \".\""
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "for column in ev_charging_traffic.columns:\n",
    "    if ev_charging_traffic[column].dtype == 'object':\n",
    "        ev_charging_traffic[column] = ev_charging_traffic[column].str.replace(',', '.')\n",
    "    \n",
    "ev_charging_traffic.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Task 7\n",
    "\n",
    "Next, convert the data types of all the columns of `ev_charging_traffic` to floats."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "ev_charging_traffic = ev_charging_traffic.astype(float)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Task Group 3 - Train Test Split\n",
    "\n",
    "Next, let's split the dataset into training and testing datasets. \n",
    "\n",
    "The training data will be used to train the model and the testing data will be used to evaluate the model."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Task 8\n",
    "\n",
    "We create two datasets from `ev_charging_traffic`:\n",
    "\n",
    "- `X` contains only the input numerical features\n",
    "- `y` contains only the target column `El_kWh`"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "numerical_features = ev_charging_traffic.drop(['El_kWh'], axis=1).columns\n",
    "X = ev_charging_traffic[numerical_features]\n",
    "\n",
    "y = ev_charging_traffic['El_kWh']"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Task 9\n",
    "\n",
    "Split `X` and `y` into training and testing datasets using sklearn. The training set should use 80% of the data. Set the `random_state` parameter to `2`."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "from sklearn.model_selection import train_test_split\n",
    "X_train, X_test, y_train, y_test = train_test_split(X, y,\n",
    "                                                    train_size=0.80,\n",
    "                                                    test_size=0.20,\n",
    "                                                    random_state=2) # set a random seed - do not modify\n",
    "\n",
    "print(\"Training size:\", X_train.shape)\n",
    "print(\"Testing size:\", X_test.shape)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Task Group 4 - Linear Regression Baseline"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "This section is optional, but useful. We want to compare our neural network to a basic linear regression. \n",
    "\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Task 10\n",
    "\n",
    "Train a Linear Regression model using the training data to predict EV charging loads.\n",
    "\n",
    "The linear regression will be used as a baseline to compare against the neural network we will train later."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "from sklearn.linear_model import LinearRegression\n",
    "\n",
    "linear_model = LinearRegression()\n",
    "linear_model.fit(X_train, y_train)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Task 11\n",
    "\n",
    "Evaluating the linear regression baseline by calculating the MSE on the testing data. Use `mean_squared_error` from `sklearn.metrics`.\n",
    "\n",
    "Save the testing MSE to the variable `test_mse` and print it out."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "from sklearn.metrics import mean_squared_error\n",
    "\n",
    "linear_test_predictions = linear_model.predict(X_test)\n",
    "test_mse = mean_squared_error(y_test, linear_test_predictions)\n",
    "print(\"Linear Regression - Test Set MSE:\", test_mse)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "The mean squared error is around `131.4`  If the square root is taken, we have about `11.5`. We can say that the linear regression, on average, is off by `11.5 kWh`."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Task Group 5 - Train a Neural Network Using PyTorch\n",
    "Let's now create a neural network using PyTorch to predict EV charging loads."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import torch\n",
    "from torch import nn\n",
    "from torch import optim"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Convert training set\n",
    "X_train_tensor = torch.tensor(X_train.values, dtype=torch.float)\n",
    "y_train_tensor = torch.tensor(y_train.values, dtype=torch.float).view(-1,1)\n",
    "\n",
    "# Convert testing set\n",
    "X_test_tensor = torch.tensor(X_test.values, dtype=torch.float)\n",
    "y_test_tensor = torch.tensor(y_test.values, dtype=torch.float).view(-1,1)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Task 14\n",
    "\n",
    "\n",
    "We create a sequential neural network with the following architecture:\n",
    "\n",
    "- input layer with number of nodes equal to the number of training features\n",
    "- a first hidden layer with `56` nodes and a ReLU activation\n",
    "- a second hidden layer with `26` nodes and a ReLU activation\n",
    "- an output layer with `1` node\n",
    "\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "\n",
    "model = nn.Sequential(\n",
    "    nn.Linear(26, 56),\n",
    "    nn.ReLU(),\n",
    "    nn.Linear(56, 26),\n",
    "    nn.ReLU(),\n",
    "    nn.Linear(26, 1)\n",
    ")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Task 15\n",
    "We define the loss function and optimizer used for training:\n",
    "\n",
    "- set the MSE loss function to the variable `loss`\n",
    "- set the Adam optimizer to the variable `optimizer` with a learning rate of `0.0007`"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "loss = nn.MSELoss()\n",
    "optimizer = optim.Adam(model.parameters(), lr=0.0007)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Task 16\n",
    "\n",
    "We create a training loop to train our neural network for 3000 epochs.\n",
    "\n",
    "We also keep track of the training loss by printing out the MSE every 500 epochs."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "num_epochs = 3000 # number of training iterations\n",
    "for epoch in range(num_epochs):\n",
    "    outputs = model(X_train_tensor) # forward pass \n",
    "    mse = loss(outputs, y_train_tensor) # calculate the loss \n",
    "    mse.backward() # backward pass\n",
    "    optimizer.step() # update the weights and biases\n",
    "    optimizer.zero_grad() # reset the gradients to zero\n",
    "\n",
    "    # keep track of the loss during training\n",
    "    if (epoch + 1) % 500 == 0:\n",
    "        print(f'Epoch [{epoch + 1}/{num_epochs}], MSE Loss: {mse.item()}')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Task 17\n",
    "\n",
    "We save the neural network in the `models` directory using the path `models/model.pth`."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "torch.save(model, 'models/model.pth')  "
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Task 18\n",
    "\n",
    "We now evaluate the neural network on the testing set. \n",
    "\n",
    "We save the testing data loss to the variable `test_loss` and use `.item()` to extract and print out the loss. "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# using the loaded neural network `loaded_model`\n",
    "model.eval() # set the model to evaluation mode\n",
    "with torch.no_grad(): # disable gradient calculations\n",
    "    predictions = model(X_test_tensor) # generate apartment rent predictions\n",
    "    test_loss = loss(predictions, y_test_tensor) # calculate testing set MSE loss\n",
    "    \n",
    "print('Neural Network - Test Set MSE:', test_loss.item()) # print testing set MSE    "
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Task 19\n",
    "\n",
    "The same model was run for 4500 epochs locally. That model is saved as `models/model4500.pth`. Load this model using PyTorch and evaluate it. How well does the longer-trained model perform?"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# load the model\n",
    "model4500 = torch.load('models/model4500.pth')\n",
    "\n",
    "# using the loaded neural network `loaded_model`\n",
    "model4500.eval() # set the model to evaluation mode\n",
    "with torch.no_grad(): # disable gradient calculations\n",
    "    predictions = model4500(X_test_tensor) # generate apartment rent predictions\n",
    "    test_loss = loss(predictions, y_test_tensor) # calculate testing set MSE loss\n",
    "    \n",
    "print('Neural Network - Test Set MSE:', test_loss.item()) # print testing set MSE"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "The increased training improved the test loss to about `115.2`. A full `12%` improvement on our linear regression baseline. So the nonlinearity introduced by the neural network actually improved the result."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.10.9"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}